In [ ]:
import branch_length
import distributions

import altair as alt
import numpy as np
import pandas as pd

alt.renderers.enable("notebook")

import importlib 
importlib.reload(branch_length)
importlib.reload(distributions)

In [ ]:
def plot_functions(f_true, f_approx):
    x_vals = np.linspace(0.1, 5, 40)
    x_transpose = np.transpose(np.array([x_vals]))
    data = pd.DataFrame({"x": x_vals, "truth": f_true(np.transpose(np.array([x_vals]))), 
                         "approx": f_approx(np.transpose(np.array([x_vals])))})
    return alt.Chart(data.melt(id_vars=["x"])).mark_line().encode(
        x='x',
        y='value',
        color='variable'
    )

In [ ]:
stepsz = 0.1
grad_clip = 100.
d = distributions.Normal(1)
true_loc = np.array([2.])
true_shape = np.array([1.])
phylo_log_like = lambda x: d.log_prob(x, true_loc, true_shape)
phylo_log_like_grad = lambda x: d.log_prob_grad(x, true_loc, true_shape)

loc = np.array([1.])
shape = np.array([0.5])

In [ ]:
x = d.sample(loc, shape, 2)
x_transpose = np.transpose(np.array([x]))
weights = branch_length.like_weights(d, phylo_log_like(x), x, loc, shape)
loc_grad, shape_grad = branch_length.param_grad(d, weights, phylo_log_like_grad(x), x, loc, shape)

print("before")
print(x)
print(weights)
print(loc_grad)
print(shape_grad)

loc += np.clip(loc_grad*stepsz, -grad_clip, grad_clip)
shape += np.clip(shape_grad*stepsz, -grad_clip, grad_clip)

print("after")
print(true_loc, loc, loc_grad*stepsz)
print(true_shape, shape, shape_grad*stepsz)

plot_functions(phylo_log_like, lambda x: d.log_prob(x, loc, shape))